In [77]:
# imports
from lxml import html, etree
import requests
import pandas as pd
import datetime as dt

In [79]:
def get_all_pages(webpage):
    
# Get the start & end of the web url around the index value
    start,end = webpage.split('&index=')
    url_start = start+'&index='
    url_end = end[1:]
    
# Initialise variables
    i=0
    j=str(i)
    result_count=1
    price_pcm=[]
    titles=[]
    addresses=[]
    weblinks=[]
    page_counts=[]
    df=pd.DataFrame(columns=['price','type','address','url'])

# Get the total number of results from the search
    page = requests.get(webpage)
    tree = html.fromstring(page.content)
    xp_result_count = '//span[@class="searchHeader-resultCount"]/text()'
    result_count = int(tree.xpath(xp_result_count)[0])
    
# Turn the total number of search results into number of iterations for the loop
    loop_count = result_count/24
    if result_count%24>0:
        loop_count = loop_count+1
        
# Set the Xpath variables for the loop
    xp_prices = '//span[@class="propertyCard-priceValue"]/text()'
    xp_titles = '//div[@class="propertyCard-details"]//a[@class="propertyCard-link"]//h2[@class="propertyCard-title"]/text()'
    xp_addresses = '//address[@class="propertyCard-address"]/text()'
    xp_weblinks = '//div[@class="propertyCard-details"]//a[@class="propertyCard-link"]/@href'

# Start the loop through the search result web pages
    for pages in range(0,loop_count,1):
        webpage = url_start+str(pages*24)+url_end
        page = requests.get(webpage)
        tree = html.fromstring(page.content)
        
# Reset variables
        price_pcm=[]
        titles=[]
        addresses=[]
        weblinks=[]

# Create data lists from Xpaths
        for val in tree.xpath(xp_prices):
            price_pcm.append(val)
        for val in tree.xpath(xp_titles):
            titles.append(val)
        for val in tree.xpath(xp_addresses):
            addresses.append(val)
        for val in tree.xpath(xp_weblinks):
            weblinks.append(val)

        data = [price_pcm, titles, addresses, weblinks]
        temp_df= pd.DataFrame(data)
        temp_df = temp_df.transpose()
        temp_df.columns=['price','type','address','url']
        temp_df = temp_df[temp_df.url != '/property-for-sale/property-0.html'] # Drop empty rows
               
        
        frames = [df,temp_df]
        
        df = pd.concat(frames)

    df = df.reset_index(drop=True)
    df.price.replace(regex=True,inplace=True,to_replace=r'\D',value=r'') # Remove non-numeric string from price col
    df.price=pd.to_numeric(df.price) # Convert price col to numeric for analysis
    now = dt.datetime.today().strftime("%d/%m/%Y")
    df['date'] = now # Add in date column
    df.to_csv('rightmove_df.csv',encoding='utf-8')
    
    return df

In [80]:
webpage = 'http://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=REGION%5E70417&numberOfPropertiesPerPage=24&radius=0.0&sortType=6&index=0&propertyTypes=detached%2Csemi-detached%2Cterraced%2Cflat%2Cbungalow&maxDaysSinceAdded=7&includeLetAgreed=false&viewType=LIST&currencyCode=GBP'

In [81]:
df=get_all_pages(webpage)

In [82]:
df.head()

,price,type,address,url,date
0,1250,1 bedroom flat,"Broadway Market, London",/property-to-rent/property-61046081.html,09/08/2016
1,1625,2 bedroom maisonette,"Graham Road, Hackney, E8",/property-to-rent/property-55565083.html,09/08/2016
2,1430,1 bedroom apartment,"Sylvester Road, Hackney, London",/property-to-rent/property-43601331.html,09/08/2016
3,1746,2 bedroom apartment,"Sylvester Path, Hackney, London",/property-to-rent/property-43600500.html,09/08/2016
4,1625,2 bedroom flat,"Sylvester Path, Hackney, London",/property-to-rent/property-51218096.html,09/08/2016


In [83]:
df.type.unique()

array(['1 bedroom flat', '2 bedroom maisonette', '1 bedroom apartment',
       '2 bedroom apartment', '2 bedroom flat', '3 bedroom apartment',
       'Studio flat', '3 bedroom terraced house',
       '4 bedroom terraced house', '1 bedroom property', '3 bedroom flat',
       '2 bedroom duplex', '1 bedroom detached house',
       '4 bedroom apartment', '2 bedroom semi-detached house'], dtype=object)

In [60]:
df.head()

,price,type,address,url
0,1250,1 bedroom flat,"Broadway Market, London",/property-to-rent/property-61046081.html
1,1625,2 bedroom maisonette,"Graham Road, Hackney, E8",/property-to-rent/property-55565083.html
2,1430,1 bedroom apartment,"Sylvester Road, Hackney, London",/property-to-rent/property-43601331.html
3,1746,2 bedroom apartment,"Sylvester Path, Hackney, London",/property-to-rent/property-43600500.html
4,1625,2 bedroom flat,"Sylvester Path, Hackney, London",/property-to-rent/property-51218096.html


In [58]:
df.dtypes

price       int64
type       object
address    object
url        object
dtype: object

In [301]:
# error checking
# method for exporting full html tyext for inspection
page = requests.get(webpage)
tree = html.fromstring(page.content)
html_text=etree.tostring(tree)
file = open("html.txt", "w")
file.write(html_text)
file.close()
xp_prices = '//span[@class="propertyCard-priceValue"]/text()'
type(tree.xpath(xp_prices))
# error checking


list